In [16]:
import pandas as pd
df = pd.read_csv('../data/flipkart_product.csv', encoding='latin1')
df['Review'] = df['Review'].astype(str).str.replace('\n', ' ').str.strip()
df['Summary'] = df['Summary'].astype(str).str.replace('\n', ' ').str.strip()
df['full_text'] = df['Summary'] + ' ' + df["Review"]

def sentiment_label(rate):
    if rate ==2 or rate < 2: return 0 #negative
    elif rate == 3: return 1 #neutral
    else: return 2 # positive

df['Rate'] = pd.to_numeric(df['Rate'], errors = 'coerce')    
df['sentiment'] = df['Rate'].apply(sentiment_label)

df['fake_label'] = ((df['full_text'].str.len() < 30) & (df['Rate'] >= 4)).astype(int)

df.to_csv('../data/cleaned.csv', index = False)

print(df.head())

                                         ProductName     Price  Rate  \
0  Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999   5.0   
1  Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999   5.0   
2  Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999   3.0   
3  Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999   1.0   
4  Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999   3.0   

            Review                                            Summary  \
0           Super!  Great cooler.. excellent air flow and for this...   
1          Awesome             Best budget 2 fit cooler. Nice cooling   
2             Fair  The quality is good but the power of air is de...   
3  Useless product                 Very bad product it's a only a fan   
4             Fair                                      Ok ok product   

                                           full_text  sentiment  fake_label  
0  Great cooler.. excellent air flow and for this.

In [17]:
from sklearn.model_selection import train_test_split

X = df['full_text'].values
y_sentiment = df['sentiment'].values
y_fake = df['fake_label'].values

X_train, X_test, y_sent_train, y_sent_test = train_test_split(X, y_sentiment, test_size=0.2, random_state=42)


In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 5000 #consider top 5000 words
max_len = 100 # max sequence length

tokenizer = Tokenizer(num_words = max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, paddings='post')

2025-09-16 13:26:29.802956: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-16 13:26:30.307341: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-16 13:26:32.710649: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


TypeError: pad_sequences() got an unexpected keyword argument 'paddings'

In [ ]:
model_sentiment = Sequential([
    Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax') #3 classes: negative, neutral, positive
])

model_sentiment.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_sentiment.summary()

NameError: name 'Sequential' is not defined

In [ ]:
history_sent = model_sentiment.fit(X_train_pad, y_sent_train, 
                                   epochs=5,
                                   batch_size=64,
                                   validation_data=(X_test_pad, y_sent_test))

NameError: name 'model_sentiment' is not defined

In [23]:
model_fake = Sequential([
    Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid') #binary classification
])

model_fake.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history_fake = model_fake.fit(X_train_pad, y_fake_train,
                              epochs = 5,
                              batch_size = 64,
                              validation_data = (X_test_pad, y_fake_test))

NameError: name 'Sequential' is not defined

In [ ]:
model_sentiment.save('../models/sentiment_model.h5')
model_fake.save('../models/fake_model.h5')